In [2]:
import urllib, http.client
import time
import json
# эти модули нужны для генерации подписи API
import hmac, hashlib
import sys
import copy
import pandas as pd
import csv
sys.path

['',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/setuptools-39.1.0-py3.5.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/home/lev/.ipython']

In [3]:
API_KEY ='K-3038939b9a19156293f232b73e6d2392c9799078'
API_SECRET = 'S-c25c5229f8be56020bcd495c8adac52e02ff0c4a'

In [4]:
CURRENCY_1 = 'BTC'
CURRENCY_2 = 'RUB'
CURRENCY_1_MIN_QUANTITY = 0.001

In [5]:
ORDER_LIFE_TIME = 3 # минуты
STOCK_FEE = 0.002 # комиссия == 0.2%
AVG_PRICE_PERIOD = 90 # за какой период брать ср цену(минуты)
CAN_SPEND = 5 # сколько тратить CUR_2 при каждой покупке CUR_1
BLOCK_SPEND = 30  # ограничение CUR_2 на покупку
PROFIT_MARKUP = 0.001 # прибыль в 1%
DEBUG = True # инфа отладки

STOCK_TIME_OFFSET = 0 # если время биржи расходится с нашим
# обращение к api
API_URL = 'api.exmo.me'
API_VERSION = 'v1'

CURRENT_PAIR = CURRENCY_1 + '_' + CURRENCY_2

In [6]:
class ExmoAPI:
#     тут всё понятно
    def __init__(self, API_KEY, API_SECRET, API_URL = 'api.exmo.me', API_VERSION = 'v1'):
        self.API_URL = API_URL
        self.API_VERSION = API_VERSION
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')
        
# Из строки payload получаем "подпись", хешируем с помощью секретного ключа API
# sing - получаемый ключ, который будет отправлен на биржу для проверки
    def sha512(self, data):
        H = hmac.new(key = self.API_SECRET, digestmod = hashlib.sha512)
        H.update(data.encode('utf-8'))
        return H.hexdigest()
    
# Составляем словарь {ключ:значение} для отправки на биржу
# пока что в нём {'nonce':2938491248715}

    def api_query(self, api_method, params = {}):
        params['nonce'] = int(round(time.time() * 1000))
        # Переводим словарь payload в строку, в формат для отправки через GET/POST и т.п.
        params =  urllib.parse.urlencode(params)

        sign = self.sha512(params)
        
        # Формируем заголовки request для отправки запроса на биржу. 
        # Передается публичный ключ API и подпись, полученная с помощью hmac
        headers = {
            "Content-type": "application/x-www-form-urlencoded",
            "Key": self.API_KEY,
            "Sign": sign
        }
# Создаем подключение к бирже, если в течении 60 сек не удалось подключиться, обрыв соединения    
        conn = http.client.HTTPSConnection(self.API_URL, timeout=60)
    # После установления связи, запрашиваем переданный адрес
    # В заголовке запроса уходят headers, в теле - payload
        conn.request("POST", "/" + self.API_VERSION + "/" + api_method, params, headers)
         # Получаем ответ с биржи и читаем его в переменную response
        response = conn.getresponse().read()
        conn.close()
        
        try:
            obj = json.loads(response.decode('utf-8'))
            # Смотрим, есть ли в полученном объекте ключ "error"
            if 'error' in obj and obj['error']:
                print(obj['error'])
                raise sys.exit()
            return obj
        # Если не удалось перевести полученный ответ (вернулся не JSON)
        except json.decoder.JSONDecodeError:
            print('Error while parsing response:', response)
            raise sys.exit()

In [8]:
api = ExmoAPI(API_KEY, API_SECRET)
# print(ExmoAPI_instance.api_query('user_info'))
for i in api.api_query('user_info').items():
    print(i)
# print(ExmoAPI_instance.api_query())

('uid', 1421646)
('server_date', 1529902680)
('reserved', {'PLN': '0', 'ETH': '0', 'EOS': '0', 'XRP': '0', 'BTC': '0', 'RUB': '20', 'XMR': '0', 'USDT': '0', 'BCH': '0', 'LTC': '0', 'WAVES': '0', 'DASH': '0', 'KICK': '0', 'UAH': '0', 'DOGE': '0', 'ZEC': '0', 'HBZ': '0', 'USD': '0', 'ETC': '0', 'BTG': '0', 'EUR': '0'})
('balances', {'PLN': '0', 'ETH': '0', 'EOS': '0', 'XRP': '0', 'BTC': '0', 'RUB': '74', 'XMR': '0', 'USDT': '0', 'BCH': '0', 'LTC': '0', 'WAVES': '0', 'DASH': '0', 'KICK': '0', 'UAH': '0', 'DOGE': '0', 'ZEC': '0', 'HBZ': '0', 'USD': '0', 'ETC': '0', 'BTG': '0', 'EUR': '0'})


In [11]:
trade_time ={}

def create_one_interval(order):
    interval = {}
    interval['total_amount']=float(order['amount'])
    interval['total_quantity']=float(order['quantity'])
    interval['min_buy_price']=None
    interval['min_sell_price']=None
    interval['max_buy_price']=0
    interval['max_sell_price']=0
    interval['total_buy_quantity']=0
    interval['total_sell_quantity']=0
    interval['bought_total_amount']=0
    interval['sell_total_amount']=0
    interval['middle_buy_price'] = 0
    interval['middle_sell_price'] = 0
    interval['middle_price'] = float(order['price'])
    
    if order['type']=='buy':
        interval['max_buy_price']=float(order['price'])
        interval['min_buy_price']=float(order['price'])
        interval['total_buy_quantity']=float(order['quantity'])
        interval['bought_total_amount']=float(order['amount'])
        interval['middle_buy_price'] = float(order['price'])
        
    elif order['type']=='sell':
        interval['max_sell_price']=float(order['price'])
        interval['min_sell_price']=float(order['price'])
        interval['total_sell_quantity']=float(order['quantity'])
        interval['sell_total_amount']=float(order['amount'])
        interval['middle_sell_price'] = float(order['price'])
        
    return interval
    
def upgrade_one_interval(order, interval):
    #     общее число        
    interval['total_quantity'] += float(order['quantity'])
    interval['total_amount'] += float(order['amount'])
    interval['middle_price'] = interval['total_amount'] / interval['total_quantity']
#     минимальная и максимальная цена покупки
    if (order['type']=='buy'):
        if float(order['price'])>interval['max_buy_price']:
            interval['max_buy_price'] = float(order['price'])
        if interval['min_buy_price'] == None:
            interval['min_buy_price'] = float(order['price'])
        elif (float(order['price'])<interval['min_buy_price']):
            interval['min_buy_price'] = float(order['price'])
#             общ стоимость и объем покупки
        interval['bought_total_amount']+=float(order['amount'])
        interval['total_buy_quantity']+=float(order['quantity'])
        interval['middle_buy_price'] = interval['bought_total_amount']/interval['total_buy_quantity']
#             минимальная и максимальная цена продажи
    elif order['type']=='sell':
        if float(order['price']) > interval['max_sell_price']:
            interval['max_sell_price'] = float(order['price'])
        if interval['min_sell_price'] == None:
            interval['min_sell_price'] = float(order['price'])
        elif float(order['price']) <interval['min_sell_price']:
            interval['min_sell_price'] = float(order['price'])
        interval['sell_total_amount'] += float(order['amount'])
        interval['total_sell_quantity'] += float(order['quantity'])
        interval['middle_sell_price'] = interval['sell_total_amount']/interval['total_sell_quantity']
#     перевес 
    if interval['bought_total_amount']>interval['sell_total_amount']:
        interval['pereves_val'] = interval['bought_total_amount']-interval['sell_total_amount']
#         interval['pereves_ps'] = ((interval['bought_total_amount'] - interval['sell_total_amount'])/ interval['sell_total_amount']) * 100 
        interval['flag'] = 1 #флаг перевеса покупки для классификации
    elif interval['sell_total_amount'] > interval['bought_total_amount']:
        interval['pereves_val'] = interval['sell_total_amount'] - interval['bought_total_amount']
#         interval['pereves_ps'] = ((interval['sell_total_amount'] - interval['bought_total_amount'])/interval['bought_total_amount']) * 100
        interval['flag'] = 0 #флаг перевеса продаж для классификации
    #  немного статистики можнео накидать(тип в % как в выводе функции ниже)
    return interval
    

    
    


for string in api.api_query('trades',{'pair':str(CURRENCY_1)+'_'+str(CURRENCY_2),'limit':100}).items():
    for order in string[1]:
#         print(order)
        if trade_time == {}:
            trade_time['%s' % (order['date'])] = create_one_interval(order)
            min_key = order['date']
        else:        
            for i in trade_time.keys():
                if int(i) < min_key:
                    min_key = int(i)            
            if (min_key - int(order['date'])) > 900: # тип на 5 минут больше
                trade_time['%s' %(order['date'])] = create_one_interval(order)
            else:
                trade_time[str(min_key)] = upgrade_one_interval(order,(trade_time[str(min_key)]))
                
            
            
df = pd.DataFrame.from_dict(trade_time)
df = df.T


df.to_csv('result.csv')
df.head()
        

,bought_total_amount,flag,max_buy_price,max_sell_price,middle_buy_price,middle_price,middle_sell_price,min_buy_price,min_sell_price,pereves_val,sell_total_amount,total_amount,total_buy_quantity,total_quantity,total_sell_quantity
1529895491,0.000000e+00,NaN,0.000000,400573.075251,0.000000,400573.075251,400573.075251,NaN,400573.075251,NaN,4.406304e+02,4.406304e+02,0.000000,0.001100,0.001100
1529896444,1.824439e+06,1.0,401281.295213,400117.000000,400609.330360,400359.928797,399926.262293,399504.337953,399504.337953,7.769933e+05,1.047446e+06,2.871885e+06,4.554160,7.173257,2.619097
1529897501,2.346590e+06,1.0,400928.000000,399710.000000,400681.243746,400457.566789,399692.034411,400256.000000,399504.337953,1.662644e+06,6.839452e+05,3.030535e+06,5.856500,7.567681,1.711181
1529898410,1.032082e+06,1.0,400820.000000,399878.000000,400652.913043,400213.309492,399752.309138,400454.000000,399489.542878,5.011517e+04,9.819667e+05,2.014049e+06,2.576000,5.032438,2.456438
1529899526,1.566689e+06,1.0,402961.969370,401752.000000,400457.481331,400688.366224,401023.590813,400400.000000,399489.542880,4.861114e+05,1.080578e+06,2.647267e+06,3.912248,6.606797,2.694549
